combine RAG and Fine-tuning

 app mode

import modules

In [1]:
from unittest import skipIf

from nbclient.jsonutil import json_clean
from openai import OpenAI
from sk import my_sk

import csv
import json
import random

create client

In [2]:
client=OpenAI(api_key=my_sk)

preparing training data (fine tuning only accept jsonl format)

In [45]:
# load csv of Youtube comment
#convert csv file into two lists: comment list and response list
comment_list = []
response_list = []

with open("data/YT-comments.csv",mode='r') as file:
     file = csv.reader(file)

     for line in file:
         #skip first line
         if line[0]=='Comment':
             continue
         comment_list.append(line[0])
         response_list.append(line[1]+"--ArielGPT")

In [9]:
len(comment_list)

59

convert comment list and response list into jsonl

In [46]:
# construct training examples into jsonl

example_list=[]

intstructions_string_few_shot = """ArielGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–-ArielGPT'. \
ArielGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging. The length of the response is at most 70 words.

Here are some examples of ArielGPT responding to viewer comments.
Viewer comment: This was a very thorough introduction to LLMs and answered many questions I had. Thank you.
ArielGPT: Great to hear, glad it was helpful :) --ArielGPT

Viewer comment: Epic, very useful for my BCI class
ArielGPT: Thanks, glad to hear!  --ArielGPT

Viewer comment: Honestly the most straightforward explanation I've ever watched. Super excellent work Ariel. Thank you. It's so rare to find good communicators like you!
ArielGPT: Thanks, glad it was clear   --ArielGPT
"""

for i in range(len(comment_list)):
    system_dict={"role":"system", "content":intstructions_string_few_shot}
    user_dict={"role":"user", "content":comment_list[i]}
    assistant_dict={"role":"assistant", "content":response_list[i]}

    message_list=[system_dict,user_dict,assistant_dict]

    example_list.append({"messages":message_list})

We separate the example_list into training data and validation data

In [47]:
validation_index_list=random.sample(range(len(example_list)),9)

validation_list=[example_list[i] for i in validation_index_list]

for example in validation_list:
    example_list.remove(example)

In [53]:
len(example_list)

50

In [54]:
len(validation_list)

9

In [48]:
with open('training-data.jsonl','w') as training_file:
    for example in example_list:
        json.dump(example,training_file)
        training_file.write("\n")

with open('validation-data.jsonl','w') as validation_file:
    for example in validation_list:
        json.dump(example,validation_file)
        validation_file.write("\n")

upload training and validation examples to openai api

In [49]:
training_file=client.files.create(
    file=open("training-data.jsonl","rb"),
    purpose="fine-tune"
)

validation_file=client.files.create(
    file=open("validation-data.jsonl","rb"),
    purpose="fine-tune"
)

create a fine-tuned model

In [50]:
client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    suffix="ArielGPT",
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-8e6WS8vM32BNJznGQF5blF5N', created_at=1745737485, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Ydl53eo2JtVEYNSQg3WFjf7Q', result_files=[], seed=2099968351, status='validating_files', trained_tokens=None, training_file='file-KFgvsjbZDLQzbcWAUuKRXE', validation_file='file-LNg3b7QieAffvc1QkWDJEv', estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix='ArielGPT')

In [60]:
# check fine-tune api status

job = client.fine_tuning.jobs.retrieve("ftjob-8e6WS8vM32BNJznGQF5blF5N")
print(job.status)
print(job.fine_tuned_model)


succeeded
ft:gpt-3.5-turbo-0125:personal:arielgpt:BQqTards


use fine-tuned model

In [36]:
#test_comment="Great content, thank you!"
test_comment = "I am typing this after watching half of the video as I am already amazed with the clarity of explanation. exceptional."
#test_comment = "What is fat-tailedness?"

In [62]:
response=client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:arielgpt:BQqTards",
    messages=[
        {"role":"system", "content":intstructions_string_few_shot},
        {"role":"user", "content":test_comment}
         ]
)

In [33]:
print(response)

ChatCompletion(id='chatcmpl-BQqD047K4752SOdh4EfSPddHt0YX5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Glad to hear, I aim to make GPT swallowable 😅ArielGPT', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1745736918, model='ft:gpt-3.5-turbo-0125:personal:arielgpt:BQq6xScU', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=266, total_tokens=286, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [28]:
print(dict(response))

{'id': 'chatcmpl-BQq7xp8tdPciuRWVffUQo1Hc5KteX', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Thanks, I’m glad you liked it!ArielGPT', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], 'created': 1745736605, 'model': 'ft:gpt-3.5-turbo-0125:personal:arielgpt:BQq6xScU', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': None, 'usage': CompletionUsage(completion_tokens=14, prompt_tokens=249, total_tokens=263, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))}


In [63]:
print(dict(response)['choices'][0].message.content)

Thanks, glad it was clear 😊--ArielGPT
